In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pypsa
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature

plt.style.use("ggplot")
%matplotlib inline

In [ ]:
regions = gpd.read_file("../input/regions_onshore_elec_s_256.geojson")

In [ ]:
regions.plot(figsize=(8, 8), column="name", cmap="tab20b", linewidth=1, edgecolor="k")

In [ ]:
n = pypsa.Network("../input/elec_s_256_ec.nc")

In [ ]:
n.generators_t.p_max_pu.filter(like="onwind").T

In [ ]:
def plot_average_capacity_factor(regions, network, carrier):
    if carrier == "onwind":
        colormap = "viridis"
    if carrier == "solar":
        colormap = "cividis"

    # Extract time series of specified carrier feed-in per region and calculate average
    carrier_data = network.generators_t.p_max_pu.filter(like=carrier).mean()

    # Adjust the renaming to include the full region and sub-region code
    capacity_factors = carrier_data.rename(lambda x: " ".join(x.split()[:-1])).to_frame(
        "average_capacity_factor"
    )

    # Merge using the updated keys
    regions = regions.merge(
        capacity_factors, left_on="name", right_index=True, how="left"
    )

    fig, ax = plt.subplots(
        subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(10, 8)
    )
    ax.set_extent([-15, 30, 35, 65], crs=ccrs.PlateCarree())

    map_opts = {
        "color_geomap": {
            "ocean": "lightblue",
            "land": "white",
            "border": "black",
            "coastline": "black",
        },
    }

    # Add map features with custom styles
    ax.add_feature(
        cfeature.OCEAN.with_scale("50m"),
        facecolor=map_opts["color_geomap"]["ocean"],
        zorder=0,
    )
    ax.add_feature(
        cfeature.LAND.with_scale("50m"),
        facecolor=map_opts["color_geomap"]["land"],
        zorder=0,
    )
    ax.add_feature(
        cfeature.BORDERS.with_scale("50m"),
        edgecolor=map_opts["color_geomap"]["border"],
        linewidth=0.5,
        alpha=0.5,
        zorder=1,
    )
    ax.add_feature(
        cfeature.COASTLINE.with_scale("50m"),
        edgecolor=map_opts["color_geomap"]["coastline"],
        linewidth=0.5,
        alpha=0.5,
        zorder=1,
    )

    regions.plot(column="average_capacity_factor", ax=ax, legend=True, cmap=colormap)

    if carrier == "onwind":
        _ = "onshore wind"
    if carrier == "solar":
        _ = "solar PV"

    ax.coastlines()
    ax.set_title(f"Annual average capacity factor for {_}")
    plt.show()


plot_average_capacity_factor(regions, n, "onwind")

In [ ]:
plot_average_capacity_factor(regions, n, "solar")

In [ ]:
def compute_pearson_correlation_falloff(network, carrier, base_region):
    """
    Computes the Pearson correlation coefficient between the p_max_pu time series
    of a specified base region for a given carrier and all other regions.

    Returns:
    - A DataFrame with each region and its correlation coefficient with the base region.
    """

    feedin = network.generators_t.p_max_pu.filter(like=carrier)

    base_series = feedin[f"{base_region} {carrier}"]

    correlations = []
    for region in feedin.columns:
        other_series = feedin[region]
        correlation = base_series.corr(other_series)

        region_name = " ".join(region.split()[:-1])
        correlations.append((region_name, correlation))

    correlation_df = pd.DataFrame(correlations, columns=["Region", "Correlation"])
    return correlation_df.sort_values("Correlation", ascending=False)


correlation_df = compute_pearson_correlation_falloff(n, "onwind", "IT1 0")
print(correlation_df)

In [ ]:
def plot_pearson_falloff(
    regions, network, carrier="onwind", base_region="DK1 0", colormap="viridis"
):
    data = compute_pearson_correlation_falloff(n, carrier, base_region)

    merged_data = regions.merge(data, left_on="name", right_on="Region", how="left")

    fig, ax = plt.subplots(
        subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(10, 8)
    )
    ax.set_extent([-15, 30, 35, 65], crs=ccrs.PlateCarree())

    map_opts = {
        "color_geomap": {
            "ocean": "lightblue",
            "land": "white",
            "border": "black",
            "coastline": "black",
        },
    }

    # Add map features with custom styles
    ax.add_feature(
        cfeature.OCEAN.with_scale("50m"),
        facecolor=map_opts["color_geomap"]["ocean"],
        zorder=0,
    )
    ax.add_feature(
        cfeature.LAND.with_scale("50m"),
        facecolor=map_opts["color_geomap"]["land"],
        zorder=0,
    )
    ax.add_feature(
        cfeature.BORDERS.with_scale("50m"),
        edgecolor=map_opts["color_geomap"]["border"],
        linewidth=0.5,
        alpha=0.5,
        zorder=1,
    )
    ax.add_feature(
        cfeature.COASTLINE.with_scale("50m"),
        edgecolor=map_opts["color_geomap"]["coastline"],
        linewidth=0.5,
        alpha=0.5,
        zorder=1,
    )

    merged_data.plot(
        column="Correlation", ax=ax, legend=True, cmap=colormap, vmin=0, vmax=1
    )

    if carrier == "onwind":
        _ = "onshore wind"
    if carrier == "solar":
        _ = "solar PV"

    ax.coastlines()
    ax.set_title(
        f"Pearson corr. falloff with distance \n"
        f"data: {_} feed-in | base region: {base_region}"
    )

    # plt.show()
    plt.savefig(
        f"../manuscript/img/pearson_corr_falloff_{carrier}_{base_region}.pdf",
        bbox_inches="tight",
    )


plot_pearson_falloff(
    regions, n, carrier="onwind", base_region="DK1 0", colormap="magma"
)

In [ ]:
plot_pearson_falloff(
    regions, n, carrier="onwind", base_region="PT1 0", colormap="magma"
)

In [ ]:
plot_pearson_falloff(regions, n, carrier="solar", base_region="DK1 0", colormap="magma")